In [13]:
# import necessary libraries
import pandas as pd

In [14]:
raw_data = pd.read_csv('../../data/covid19_us_counties.csv')

## Data preparation
### Tidy data 

In [15]:
# sample data
raw_data.head()

,Admin 2 FIPS Code;Province/State;Admin 2 Level (City/County/Borough/Region);Date;Total Death;Total Confirmed;location;Year
36103.0;New York;Suffolk;2020-06-22;1965;41010;40.88320119,-72.8012172;2020
36105.0;New York;Sullivan;2020-06-22;45;1438;41.71579493,-74.76394559;2020
37133.0;North Carolina;Onslow;2020-06-22;3;221;34.72607366,-77.42908179;2020
38001.0;North Dakota;Adams;2020-06-22;0;0;46.09686891,-102.5285397;2020
39063.0;Ohio;Hancock;2020-06-22;5;63;41.00250487,-83.66838948;2020


In [16]:
# how to make right?
raw_data = pd.read_csv('../../data/covid19_us_counties.csv', sep=';')

In [17]:
raw_data.head() # This is better.

,Admin 2 FIPS Code,Province/State,Admin 2 Level (City/County/Borough/Region),Date,Total Death,Total Confirmed,location,Year
0,36103.0,New York,Suffolk,2020-06-22,1965,41010,"40.88320119,-72.8012172",2020
1,36105.0,New York,Sullivan,2020-06-22,45,1438,"41.71579493,-74.76394559",2020
2,37133.0,North Carolina,Onslow,2020-06-22,3,221,"34.72607366,-77.42908179",2020
3,38001.0,North Dakota,Adams,2020-06-22,0,0,"46.09686891,-102.5285397",2020
4,39063.0,Ohio,Hancock,2020-06-22,5,63,"41.00250487,-83.66838948",2020


## Header name ?

In [18]:
raw_data.columns

Index(['Admin 2 FIPS Code', 'Province/State',
       'Admin 2 Level (City/County/Borough/Region)', 'Date', 'Total Death',
       'Total Confirmed', 'location', 'Year'],
      dtype='object')

In [19]:
# US doesn't have a province, but states
# FIPS code only, Admin 2 Level is not necessary
# What is total confirmed?
raw_data.rename(columns={
    'Admin 2 FIPS Code': 'FIPS',
    'Province/State': 'State',
    'Admin 2 Level (City/County/Borough/Region)': 'City/County/Borough/Region',
    'Total Confirmed': 'infected total',
    'Total Death': 'death total',
}, inplace=True)


In [20]:
raw_data.head()

,FIPS,State,City/County/Borough/Region,Date,death total,infected total,location,Year
0,36103.0,New York,Suffolk,2020-06-22,1965,41010,"40.88320119,-72.8012172",2020
1,36105.0,New York,Sullivan,2020-06-22,45,1438,"41.71579493,-74.76394559",2020
2,37133.0,North Carolina,Onslow,2020-06-22,3,221,"34.72607366,-77.42908179",2020
3,38001.0,North Dakota,Adams,2020-06-22,0,0,"46.09686891,-102.5285397",2020
4,39063.0,Ohio,Hancock,2020-06-22,5,63,"41.00250487,-83.66838948",2020


### Remove duplicates 

In [21]:
# check duplicates
raw_data.duplicated().sum() # no duplicates, good!

0

### Homogeneous variables

In [22]:
import json

In [23]:
name_of_state_us = json.load(open('name_state_us.json'))

In [24]:
raw_data['Abbr'] = raw_data['State'].map(name_of_state_us)


In [25]:
raw_data.head()

,FIPS,State,City/County/Borough/Region,Date,death total,infected total,location,Year,Abbr
0,36103.0,New York,Suffolk,2020-06-22,1965,41010,"40.88320119,-72.8012172",2020,NY
1,36105.0,New York,Sullivan,2020-06-22,45,1438,"41.71579493,-74.76394559",2020,NY
2,37133.0,North Carolina,Onslow,2020-06-22,3,221,"34.72607366,-77.42908179",2020,NC
3,38001.0,North Dakota,Adams,2020-06-22,0,0,"46.09686891,-102.5285397",2020,ND
4,39063.0,Ohio,Hancock,2020-06-22,5,63,"41.00250487,-83.66838948",2020,OH


In [26]:
raw_data['Abbr'].isnull().sum() # something went wrong?

63510

In [27]:
# drop missing 'Abbr' (not a US State)
raw_data = raw_data.dropna(subset=['Abbr'])

In [28]:
raw_data['Abbr'].isnull().sum()

0

In [29]:
# locate must show as (lat, lon)
raw_data[['lat', 'lon']] = raw_data['location'].str.split(',', expand=True)
raw_data.drop('location', axis=1, inplace=True)

In [30]:
raw_data.head()

,FIPS,State,City/County/Borough/Region,Date,death total,infected total,Year,Abbr,lat,lon
0,36103.0,New York,Suffolk,2020-06-22,1965,41010,2020,NY,40.88320119,-72.8012172
1,36105.0,New York,Sullivan,2020-06-22,45,1438,2020,NY,41.71579493,-74.76394559
2,37133.0,North Carolina,Onslow,2020-06-22,3,221,2020,NC,34.72607366,-77.42908179
3,38001.0,North Dakota,Adams,2020-06-22,0,0,2020,ND,46.09686891,-102.5285397
4,39063.0,Ohio,Hancock,2020-06-22,5,63,2020,OH,41.00250487,-83.66838948


### Unique identifier

In [31]:
# unique identifier is Date + index of row
raw_data['id'] = raw_data['Abbr'] + raw_data.index.astype(str)  

In [32]:
raw_data.head() # id is smell, but it's ok for now.

,FIPS,State,City/County/Borough/Region,Date,death total,infected total,Year,Abbr,lat,lon,id
0,36103.0,New York,Suffolk,2020-06-22,1965,41010,2020,NY,40.88320119,-72.8012172,NY0
1,36105.0,New York,Sullivan,2020-06-22,45,1438,2020,NY,41.71579493,-74.76394559,NY1
2,37133.0,North Carolina,Onslow,2020-06-22,3,221,2020,NC,34.72607366,-77.42908179,NC2
3,38001.0,North Dakota,Adams,2020-06-22,0,0,2020,ND,46.09686891,-102.5285397,ND3
4,39063.0,Ohio,Hancock,2020-06-22,5,63,2020,OH,41.00250487,-83.66838948,OH4


### Data type

In [33]:
raw_data.dtypes

FIPS                          float64
State                          object
City/County/Borough/Region     object
Date                           object
death total                     int64
infected total                  int64
Year                            int64
Abbr                           object
lat                            object
lon                            object
id                             object
dtype: object

In [34]:
# First of all FIPS is not float, it's string
raw_data['FIPS'] = raw_data['FIPS'].astype(str).str.replace('.0', '')

# Date is not a string, it's a date
raw_data['Date'] = pd.to_datetime(raw_data['Date'])

# Lat and Lon are not string, they are floated
raw_data['lat'] = raw_data['lat'].astype(float)
raw_data['lon'] = raw_data['lon'].astype(float)

In [35]:
raw_data.head()

,FIPS,State,City/County/Borough/Region,Date,death total,infected total,Year,Abbr,lat,lon,id
0,36103,New York,Suffolk,2020-06-22,1965,41010,2020,NY,40.883201,-72.801217,NY0
1,36105,New York,Sullivan,2020-06-22,45,1438,2020,NY,41.715795,-74.763946,NY1
2,37133,North Carolina,Onslow,2020-06-22,3,221,2020,NC,34.726074,-77.429082,NC2
3,38001,North Dakota,Adams,2020-06-22,0,0,2020,ND,46.096869,-102.528540,ND3
4,39063,Ohio,Hancock,2020-06-22,5,63,2020,OH,41.002505,-83.668389,OH4


In [36]:
raw_data.dtypes

FIPS                                  object
State                                 object
City/County/Borough/Region            object
Date                          datetime64[ns]
death total                            int64
infected total                         int64
Year                                   int64
Abbr                                  object
lat                                  float64
lon                                  float64
id                                    object
dtype: object

In [37]:
length = raw_data.shape[0]
length = len(str(length))


In [38]:
# 
raw_data['id'] = raw_data['Abbr'] + raw_data.index.astype(str).str.zfill(length)

In [39]:
raw_data.head()

,FIPS,State,City/County/Borough/Region,Date,death total,infected total,Year,Abbr,lat,lon,id
0,36103,New York,Suffolk,2020-06-22,1965,41010,2020,NY,40.883201,-72.801217,NY0000000
1,36105,New York,Sullivan,2020-06-22,45,1438,2020,NY,41.715795,-74.763946,NY0000001
2,37133,North Carolina,Onslow,2020-06-22,3,221,2020,NC,34.726074,-77.429082,NC0000002
3,38001,North Dakota,Adams,2020-06-22,0,0,2020,ND,46.096869,-102.528540,ND0000003
4,39063,Ohio,Hancock,2020-06-22,5,63,2020,OH,41.002505,-83.668389,OH0000004


In [40]:
# show missing values
raw_data.isnull().sum()

FIPS                          0
State                         0
City/County/Borough/Region    0
Date                          0
death total                   0
infected total                0
Year                          0
Abbr                          0
lat                           0
lon                           0
id                            0
dtype: int64

In [41]:
raw_data.set_index('id', inplace=True)
raw_data

,FIPS,State,City/County/Borough/Region,Date,death total,infected total,Year,Abbr,lat,lon
id,,,,,,,,,,
NY0000000,36103,New York,Suffolk,2020-06-22,1965,41010,2020,NY,40.883201,-72.801217
NY0000001,36105,New York,Sullivan,2020-06-22,45,1438,2020,NY,41.715795,-74.763946
NC0000002,37133,North Carolina,Onslow,2020-06-22,3,221,2020,NC,34.726074,-77.429082
ND0000003,38001,North Dakota,Adams,2020-06-22,0,0,2020,ND,46.096869,-102.528540
OH0000004,39063,Ohio,Hancock,2020-06-22,5,63,2020,OH,41.002505,-83.668389
...,...,...,...,...,...,...,...,...,...,...
UT2401695,49037,Utah,San Juan,2022-02-22,47,3916,2022,UT,37.627630,-109.803371
UT2401696,49039,Utah,Sanpete,2022-02-22,0,0,2022,UT,39.372319,-111.575868
UT2401697,nan,Utah,Southwest Utah,2022-02-22,585,62930,2022,UT,37.854472,-111.441876


In [42]:
new_cols = ["Date", "FIPS","lat","lon","State","Abbr","City/County/Borough/Region","infected total","death total"]
raw_data=raw_data.reindex(columns=new_cols)
raw_data

,Date,FIPS,lat,lon,State,Abbr,City/County/Borough/Region,infected total,death total
id,,,,,,,,,
NY0000000,2020-06-22,36103,40.883201,-72.801217,New York,NY,Suffolk,41010,1965
NY0000001,2020-06-22,36105,41.715795,-74.763946,New York,NY,Sullivan,1438,45
NC0000002,2020-06-22,37133,34.726074,-77.429082,North Carolina,NC,Onslow,221,3
ND0000003,2020-06-22,38001,46.096869,-102.528540,North Dakota,ND,Adams,0,0
OH0000004,2020-06-22,39063,41.002505,-83.668389,Ohio,OH,Hancock,63,5
...,...,...,...,...,...,...,...,...,...
UT2401695,2022-02-22,49037,37.627630,-109.803371,Utah,UT,San Juan,3916,47
UT2401696,2022-02-22,49039,39.372319,-111.575868,Utah,UT,Sanpete,0,0
UT2401697,2022-02-22,nan,37.854472,-111.441876,Utah,UT,Southwest Utah,62930,585


# Conclusion before handling missing values

In [43]:
# Save data
raw_data.to_csv('../../data/covid19_usa.csv')